I am tasked with working for an investor who specializes in purchasing undervalued assets. The investor wants to perform detailed data analysis to identify opportunities for growth and profitability in a potential purchase—TellCo, a mobile service provider in the Republic of Pefkakia. my goal is to analyze customer data, provide insights, and make recommendations on whether TellCo is worth buying or selling. The analysis will be presented through a web-based dashboard and a written report.
Task 2: User Engagement Analysis
Objective: Assess user engagement using metrics such as session frequency, duration, and total traffic.

Task 4: Satisfaction Analysis
Objective: Analyze customer satisfaction based on engagement and experience.

Assign engagement and experience scores to each user using Euclidean distance.

Calculate satisfaction scores and report top satisfied customers.

Build a regression model to predict satisfaction scores.

Perform k-means clustering on engagement and experience scores.

Aggregate and report satisfaction and experience scores per cluster.

Export final user data with scores to a local MySQL database and report a screenshot of the query output

In [3]:
import sys 
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
from dotenv import load_dotenv
from sqlalchemy import create_engine
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
os.chdir('..')
sys.path.append(os.getcwd())
#from src.Eda import handle_missing_values, remove_outliers_iqr
#from src.Eda import missing_values_table, convert_bytes_to_megabytes

In [4]:
from scripts.DB_connection import PostgresConnection

# Establishing the database connection
db = PostgresConnection()
db.connect()

if db.conn:
    # Example query
    query = "SELECT * FROM xdr_data"
    result = db.execute_query(query)

    if result:
        # Convert the result to a Pandas DataFrame
        df = pd.DataFrame(result, columns=[desc[0] for desc in db.cursor.description])
        print(df.head())  # Display the first few rows of the DataFrame
    else:
        print("No results returned from the query.")
    
    # Close the connection when done
    db.close_connection()
else:
    print("Error: No database connection.")

Connected to PostgreSQL database!
      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0  9.16456699548519E+015  ...          15854611.0           2501332.0   
1                L77566A  ...         

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

lest use deferent approch instade ofusing sime cols instruction

In [5]:

from src.Eda import handle_outliers_iqr

# Assuming df is your DataFrame

# Display the number of rows before outlier removal
print(f"Number of rows before outlier removal: {df.shape[0]}")

# Apply the function to handle outliers
df_cleaned, outlier_info = handle_outliers_iqr(df)

# Display the number of rows after outlier removal
print(f"Number of rows after outlier removal: {df_cleaned.shape[0]}")

# Display information about outliers
total_outliers_removed = sum(outlier_info.values())
print(f"\nTotal number of outliers removed: {total_outliers_removed}")

# Store the cleaned DataFrame in a new DataFrame
df_outliers_removed = df_cleaned.copy()

# Optionally, display the cleaned DataFrame's first few rows
print("\nFirst few rows of the DataFrame with outliers removed:")
print(df_outliers_removed.head())


Number of rows before outlier removal: 150001
Number of rows after outlier removal: 150001

Total number of outliers removed: 451082

First few rows of the DataFrame with outliers removed:
      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0    86399.0  2.082014e+14   3.366496e+10  3.552121e+13   
1    86399.0  2.082019e+14   3.368185e+10  3.579401e+13   
2    86399.0  2.082015e+14   3.366371e+10  3.528151e+13   
3    86399.0  2.082014e+14   3.366371e+10  3.535661e+13   
4    86399.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...

In [6]:
import sys
import os
import pandas as pd

# Change the working directory to the project root if necessary
os.chdir('..')

# Add the project root to the system path
sys.path.append(os.getcwd())

# Import the necessary functions from eda.py
from src.Eda import handle_outliers_iqr, handle_missing_values, remove_duplicates

# Assuming df is your DataFrame

# Display the number of rows before outlier removal
print(f"Number of rows before outlier removal: {df.shape[0]}")

# Apply the function to handle outliers
df_cleaned, outlier_info = handle_outliers_iqr(df)

# Display the number of rows after outlier removal
print(f"Number of rows after outlier removal: {df_cleaned.shape[0]}")

# Display information about outliers
total_outliers_removed = sum(outlier_info.values())
print(f"\nTotal number of outliers removed: {total_outliers_removed}")

# Store the cleaned DataFrame in a new DataFrame
df_outliers_removed = df_cleaned.copy()

# Optionally, display the cleaned DataFrame's first few rows
print("\nFirst few rows of the DataFrame with outliers removed:")
print(df_outliers_removed.head())

# Handle missing values
df_filled = handle_missing_values(df_outliers_removed)

# Display information about missing values after handling
print("Missing values after imputation:")
print(df_filled.isnull().sum())

# Store the results in a new DataFrame
df_missing_values_handled = df_filled.copy()

# Optionally, display the first few rows of the new DataFrame
print("\nFirst few rows of the DataFrame with missing values handled:")
print(df_missing_values_handled.head())

# Remove duplicates and ensure consistency
df_consistent = remove_duplicates(df_missing_values_handled)

# Display information about the cleaned DataFrame
print("DataFrame info after ensuring consistency:")
print(df_consistent.info())

# Optionally, display the first few rows of the cleaned DataFrame
print("\nFirst few rows of the DataFrame after ensuring consistency:")
print(df_consistent.head())


Number of rows before outlier removal: 150001
Number of rows after outlier removal: 150001

Total number of outliers removed: 451082

First few rows of the DataFrame with outliers removed:
      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0    86399.0  2.082014e+14   3.366496e+10  3.552121e+13   
1    86399.0  2.082019e+14   3.368185e+10  3.579401e+13   
2    86399.0  2.082015e+14   3.366371e+10  3.528151e+13   
3    86399.0  2.082014e+14   3.366371e+10  3.535661e+13   
4    86399.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...

In [7]:


# Import the necessary function from eda.py
from src.Eda import handle_missing_values

# Assuming df_outliers_removed is your DataFrame after outlier removal

# Apply the function to handle missing values
df_filled = handle_missing_values(df_outliers_removed)

# Display information about missing values after handling
print("Missing values after imputation:")
print(df_filled.isnull().sum())

# Store the results in a new DataFrame
df_missing_values_handled = df_filled.copy()

# Optionally, display the first few rows of the new DataFrame
print("\nFirst few rows of the DataFrame with missing values handled:")
print(df_missing_values_handled.head())


Missing values after imputation:
Bearer Id                                   0
Start                                       0
Start ms                                    0
End                                         0
End ms                                      0
Dur. (ms)                                   0
IMSI                                        0
MSISDN/Number                               0
IMEI                                        0
Last Location Name                          0
Avg RTT DL (ms)                             0
Avg RTT UL (ms)                             0
Avg Bearer TP DL (kbps)                     0
Avg Bearer TP UL (kbps)                     0
TCP DL Retrans. Vol (Bytes)                 0
TCP UL Retrans. Vol (Bytes)                 0
DL TP < 50 Kbps (%)                         0
50 Kbps < DL TP < 250 Kbps (%)              0
250 Kbps < DL TP < 1 Mbps (%)               0
DL TP > 1 Mbps (%)                          0
UL TP < 10 Kbps (%)                         0
1

In [8]:

from src.Eda import remove_duplicates

# Assuming df_missing_values_handled is your DataFrame after handling missing values

# Apply the function to ensure consistency
df_cleaned = remove_duplicates(df_missing_values_handled)

# Optionally, display information about the cleaned DataFrame
print("DataFrame info after ensuring consistency:")
print(df_cleaned.info())

# Optionally, display the first few rows of the cleaned DataFrame
print("\nFirst few rows of the DataFrame after ensuring consistency:")
print(df_cleaned.head())


DataFrame info after ensuring consistency:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 150001 non-null  float64
 1   Start                                     150001 non-null  object 
 2   Start ms                                  150001 non-null  float64
 3   End                                       150001 non-null  object 
 4   End ms                                    150001 non-null  float64
 5   Dur. (ms)                                 150001 non-null  float64
 6   IMSI                                      150001 non-null  float64
 7   MSISDN/Number                             150001 non-null  float64
 8   IMEI                                      150001 non-null  float64
 9   Last Location Name                        150001 

In [14]:
#Final overview after the data has been cleaned.

import pandas as pd

def display_dataset_characteristics(df):
    print("### Dataset Characteristics ###\n")
    
    # Display the shape of the dataset
    print(f"Shape of the DataFrame: {df.shape}")
    
    # Display the column data types
    print("\nData Types of Each Column:")
    print(df.dtypes)
    
    # Display summary statistics for numeric columns
    print("\nSummary Statistics for Numeric Columns:")
    print(df.describe())
    
    # Display the count of missing values per column
    print("\nCount of Missing Values per Column:")
    print(df.isnull().sum())
    
    # Display the count of unique values per column
    print("\nCount of Unique Values per Column:")
    print(df.nunique())
    
    # Display the first few rows of the cleaned dataset
    print("\nFirst Few Rows of the DataFrame:")
    print(df.head())
    
# Apply the function to the df_cleaned DataFrame
display_dataset_characteristics(df_cleaned)

### Dataset Characteristics ###

Shape of the DataFrame: (150001, 55)

Data Types of Each Column:
Bearer Id                                   float64
Start                                        object
Start ms                                    float64
End                                          object
End ms                                      float64
Dur. (ms)                                   float64
IMSI                                        float64
MSISDN/Number                               float64
IMEI                                        float64
Last Location Name                           object
Avg RTT DL (ms)                             float64
Avg RTT UL (ms)                             float64
Avg Bearer TP DL (kbps)                     float64
Avg Bearer TP UL (kbps)                     float64
TCP DL Retrans. Vol (Bytes)                 float64
TCP UL Retrans. Vol (Bytes)                 float64
DL TP < 50 Kbps (%)                         float64
50 Kbps < DL TP < 

In [17]:
#Clustering
#In this stage we are going to reuse the results for engagement and experience segementation(k-means clustering) and assign cluster labels to users for engagement and experience.
import sys
import os
import pandas as pd
import numpy as np

# Change the working directory to the project root if necessary
os.chdir('..')

# Add the project root to the system path
sys.path.append(os.getcwd())

# Import the necessary functions from cluster.py
from src.Cluster import load_centroids, calculate_engagement_score, calculate_experience_score, save_scores

# Assuming df_cleaned is already loaded
user_data = df_cleaned.copy()

# Load the engagement and experience centroids
centroid_engagement, centroid_experience = load_centroids('centroid_engagement.csv', 'centroid_experience.csv')

# Engagement Score Calculation
# Extract the engagement-related features (session frequency, session duration, total traffic)
engagement_columns = ['Session Frequency', 'Session Duration', 'Total Traffic (Bytes)']

# Ensure we have session frequency and session duration calculated for each user
# Calculate session frequency
session_frequency = user_data.groupby('MSISDN/Number').size().reset_index(name='Session Frequency')
user_data = user_data.merge(session_frequency, on='MSISDN/Number', how='left')

# Calculate session duration (sum of session durations per user)
session_duration = user_data.groupby('MSISDN/Number')['Dur. (ms)'].sum().reset_index(name='Session Duration')
user_data = user_data.merge(session_duration, on='MSISDN/Number', how='left')

# Calculate total traffic (sum of upload and download data per user)
user_data['Total Traffic (Bytes)'] = user_data['Total UL (Bytes)'] + user_data['Total DL (Bytes)']

# Calculate the engagement score
user_data = calculate_engagement_score(user_data, centroid_engagement, engagement_columns)

# Experience Score Calculation
# Extract the experience-related features
experience_columns = ['Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)',
                        'Avg Bearer TP UL (kbps)', 'TCP DL Retrans. Vol (Bytes)',
                            'TCP UL Retrans. Vol (Bytes)']

# Calculate the experience score
user_data = calculate_experience_score(user_data, centroid_experience, experience_columns)

# Assign the calculated scores back to df_cleaned
df_cleaned['Engagement Score'] = user_data['Engagement Score']
df_cleaned['Experience Score'] = user_data['Experience Score']

# File path where the output will be saved
output_file_path = 'engagement_experience_scores.csv'

# Save the DataFrame with the Engagement and Experience scores to a CSV file
full_file_path = save_scores(df_cleaned, output_file_path)

# Print the full path of the file
print(f"The file with Engagement and Experience scores is saved at: {full_file_path}")

# Print the head of the file (first 5 rows)
print("\nThe first 5 rows of the file are:")
print(df_cleaned.head())

# Show the newly calculated Engagement and Experience scores
print("\nNewly calculated Engagement and Experience Scores:")
print(df_cleaned[['MSISDN/Number', 'Engagement Score', 'Experience Score']].head())


FileNotFoundError: [Errno 2] No such file or directory: 'centroid_engagement.csv'